# Exploração da Camada Bronze

Este notebook explora os dados da camada bronze com o objetivo de compreendê-los e propor uma tabela unificada e harmonizada.

In [1]:
from pyspark.sql import SparkSession, functions as F

spark = SparkSession.builder.appName("Exploração de Dados - Camada Bronze").getOrCreate()


In [2]:
df_yellow = spark.read.table("bronze_db.nyc_taxi_data_yellow")
df_yellow.show()

+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+--------------------+------------------+
|vendorid|tpep_pickup_datetime|tpep_dropoff_datetime|passenger_count|trip_distance|ratecodeid|store_and_fwd_flag|pulocationid|dolocationid|payment_type|fare_amount|extra|mta_tax|tip_amount|tolls_amount|improvement_surcharge|total_amount|congestion_surcharge|airport_fee|  data_hora_ingestao|ano_mes_referencia|
+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+--------------------+------------------+
|     1.0| 2023-04-01 00:14:49|  2023-04-01 00:45:01|            2.

In [3]:
df_green = spark.read.table("bronze_db.nyc_taxi_data_green")
df_green.show()

+--------+--------------------+---------------------+------------------+----------+------------+------------+---------------+-------------+-----------+-----+-------+----------+------------+---------+---------------------+------------+------------+---------+--------------------+--------------------+------------------+
|vendorid|lpep_pickup_datetime|lpep_dropoff_datetime|store_and_fwd_flag|ratecodeid|pulocationid|dolocationid|passenger_count|trip_distance|fare_amount|extra|mta_tax|tip_amount|tolls_amount|ehail_fee|improvement_surcharge|total_amount|payment_type|trip_type|congestion_surcharge|  data_hora_ingestao|ano_mes_referencia|
+--------+--------------------+---------------------+------------------+----------+------------+------------+---------------+-------------+-----------+-----+-------+----------+------------+---------+---------------------+------------+------------+---------+--------------------+--------------------+------------------+
|     2.0| 2023-05-01 00:52:10|  2023-05-01

# Transformações
Na célula abaixo temos um protótipo do ETL responsável por carregar a tabela silver "tb_corrida_taxi_ny". 
Todas as transformações de dados tiveram como base a seguinte documentação: https://www.nyc.gov/assets/tlc/downloads/pdf/data_dictionary_trip_records_yellow.pdf e https://www.nyc.gov/assets/tlc/downloads/pdf/data_dictionary_trip_records_yellow.pdf

In [4]:
df_yellow_tranformed = df_yellow.selectExpr(
    "vendorid as id_fornecedor",
    "passenger_count as quantidade_passageiros",
    "total_amount as valor_corrida",
    "tpep_pickup_datetime as data_hora_embarque",
    "tpep_dropoff_datetime as data_hora_desembarque",
    "payment_type as id_tipo_pagamento", 
    "ano_mes_referencia",
    '"YELLOW" as tipo_servico',
)

df_green_tranformed = df_green.selectExpr(
    "vendorid as id_fornecedor",
    "passenger_count as quantidade_passageiros",
    "total_amount as valor_corrida",
    "lpep_pickup_datetime as data_hora_embarque",
    "lpep_dropoff_datetime as data_hora_desembarque",
    "payment_type as id_tipo_pagamento", 
    "ano_mes_referencia",
    '"GREEN" as tipo_servico',
)

df = df_yellow_tranformed.union(df_green_tranformed)

df = (
    df.withColumn(
        "nome_fornecedor", 
        F.when(F.col("id_fornecedor") == 1, "Creative Mobile Technologies, LLC")
        .when(F.col("id_fornecedor") == 2, "Curb Mobility, LLC")
        .when(F.col("id_fornecedor") == 6, "Myle Technologies Inc")
        .when(F.col("id_fornecedor") == 7, "Helix")
        .otherwise("FORNECEDOR NÃO IDENTIFICADO")
    )
)

df = (
    df.withColumn(
        "descricao_tipo_pagamento", 
        F.when(F.col("id_tipo_pagamento") == 0, "Viagem com tarifa flexível")
        .when(F.col("id_tipo_pagamento") == 1, "Cartão de crédito")
        .when(F.col("id_tipo_pagamento") == 2, "Dinheiro")
        .when(F.col("id_tipo_pagamento") == 3, "Sem cobrança")
        .when(F.col("id_tipo_pagamento") == 4, "Contestação")
        .when(F.col("id_tipo_pagamento") == 5, "Desconhecido")
        .when(F.col("id_tipo_pagamento") == 6, "Viagem cancelada")
        .otherwise("TIPO DE PAGAMENTO NÃO IDENTIFICADO")
    )
)

df = df.withColumn("indicador_cancelamento", F.when(F.col("id_tipo_pagamento") == 6, 'S').otherwise('N'))
df = df.withColumn("indicador_viagem_sem_cobranca", F.when(F.col("id_tipo_pagamento") == 3, 'S').otherwise('N'))
df = df.withColumn("id", F.expr("uuid()"))

df_viagem_taxi_ny = df.select(
    "id",
    "id_fornecedor",
    "nome_fornecedor",
    "quantidade_passageiros",
    "valor_corrida",
    "data_hora_embarque",
    "data_hora_desembarque",
    "indicador_cancelamento",
    "indicador_viagem_sem_cobranca",
    "id_tipo_pagamento",
    "descricao_tipo_pagamento",
    "tipo_servico",
    "ano_mes_referencia",
)

df_viagem_taxi_ny.show()




+--------------------+-------------+--------------------+----------------------+-------------+-------------------+---------------------+----------------------+-----------------------------+-----------------+------------------------+------------+------------------+
|                  id|id_fornecedor|     nome_fornecedor|quantidade_passageiros|valor_corrida| data_hora_embarque|data_hora_desembarque|indicador_cancelamento|indicador_viagem_sem_cobranca|id_tipo_pagamento|descricao_tipo_pagamento|tipo_servico|ano_mes_referencia|
+--------------------+-------------+--------------------+----------------------+-------------+-------------------+---------------------+----------------------+-----------------------------+-----------------+------------------------+------------+------------------+
|e4c53a42-f242-4e1...|          1.0|Creative Mobile T...|                   0.0|        51.65|2023-05-01 00:33:13|  2023-05-01 00:53:01|                     N|                            N|              1.

In [5]:
df_viagem_taxi_ny.printSchema()

root
 |-- id: string (nullable = false)
 |-- id_fornecedor: double (nullable = true)
 |-- nome_fornecedor: string (nullable = false)
 |-- quantidade_passageiros: double (nullable = true)
 |-- valor_corrida: double (nullable = true)
 |-- data_hora_embarque: timestamp (nullable = true)
 |-- data_hora_desembarque: timestamp (nullable = true)
 |-- indicador_cancelamento: string (nullable = false)
 |-- indicador_viagem_sem_cobranca: string (nullable = false)
 |-- id_tipo_pagamento: double (nullable = true)
 |-- descricao_tipo_pagamento: string (nullable = false)
 |-- tipo_servico: string (nullable = false)
 |-- ano_mes_referencia: string (nullable = true)

